# 결측치 처리하기

ML 모델을 개발하다 보면, 데이터에서 **결측치**를 흔하게 보게 됩니다. 학습을 위해 결측치를 적당한 값으로 대치하는 것은 매우 중요하며, 여러 방법들이 있습니다.

## 비효율적인 코드 처리

다음은 전처리기 내에, 결측치 처리 함수를 정의하는 코드입니다.

In [26]:
import pandas as pd

class BadDataPreprocessor:
    def __init__(self, data: pd.DataFrame):
        self.data = data.copy()

    def impute_missing(self, column: int, method: str):
        """결측치 처리기
        
        입력된 인자 `method`에 따라 결측치를 적당히 대치합니다.
        """

        if method == "mean":
            # 평균으로 대치하기
            fill_value = self.data[column].mean()
            self.data[column] = self.data[column].fillna(fill_value)
            print(f"[DataPreprocessor]: 결측치를 '평균'({fill_value})으로 대치했습니다.")

        elif method == "median":
            # 줒앙값으로 대치하기
            fill_value = self.data[column].median()
            self.data[column] = self.data[column].fillna(fill_value)
            print(f"[DataPreprocessor]: 결측치를 '중앙값'({fill_value})으로 대치했습니다.")

        elif method == "zero":
            # 상수(0)으로 대치하기
            fill_value = 0
            self.data[column] = self.data[column].fillna(fill_value)
            print(f"[DataPreprocessor]: 결측치를 '상수'({fill_value})로 대치했습니다.")
        
        else:
            print("[DataPreprocessor]: 지원하지 않는 방식입니다.")

위 코드에서 만약에 새로운 방법이 필요한 경우: 예를 들어 최빈값으로 방법을 사용하고 싶다면, 위의 코드 자체를 수정해야 합니다. 방법이 수십가지가 되면 결측치 처리 코드가 길어지고 지나치게 복잡해집니다!

이러한 경우 **전략 패턴**을 사용하면 보다 효율적인 코드를 작성할 수 있습니다. 전략 패턴에서는 결측치 처리라는 **행동(알고리즘)**을 독립적인 **전략 클래스**로 분리합니다.

In [ ]:
class ImputationStrategy:
    def impute(self, series: pd.Series) -> pd.Series:
        ...

그리고 이런 전략을 **소유**하는 **컨텍스트 클래스**에서 전략을 주입받아 사용합니다.

In [ ]:
class BetterDataPreprocessor:
    def __init__(self, data: pd.DataFrame, imputation_strategy: ImputationStrategy):
        self.data = data.copy()
        self._imputation_strategy = imputation_strategy

    def set_imputation_strategy(self, strategy: ImputationStrategy):
        self._imputation_strategy = strategy

    def impute(self, column: int):
        self.data[column] = self._imputation_strategy.impute(self.data[column])

이제는 새로운 결측치 처리 전략을 만들고 싶으면, 직접 클래스를 구성하면 됩니다!

In [29]:
class MeanImputationStrategy(ImputationStrategy):
    def impute(self, series: pd.DataFrame):
        fill_value = series.mean()
        series = series.fillna(fill_value)
        print(f"[DataPreprocessor]: 결측치를 '평균'({fill_value})으로 대치했습니다.")
        return series

실제로는 다음과 같이 실행하게 됩니다.

In [30]:
import numpy as np

def run():
    sample_data = pd.DataFrame({'age': [25, 30, np.nan, 35, 40], 'salary': [50000, 60000, 55000, np.nan, 70000]})
    processor = BetterDataPreprocessor(sample_data, MeanImputationStrategy())
    processor.impute('age')

if __name__ == "__main__":
    run()

[DataPreprocessor]: 결측치를 '평균'(32.5)으로 대치했습니다.


## 과제

위의 `BetterDatapreprocessor`는 `ImputationaStrategy`에 같은 `row`의 다른 값을 제공하지 않으므로 `KNN Imputer`를 허용하지 못합니다. `KNN Imputer`에 대해 조사한 후, 이를 포함할 수 있는 전략 컨텍스트를 설계하고, `KNN Imputer`를 적용시켜보세요.

In [ ]:
class DataPreprocessor:
    """KNN Imputer까지 커버할 수 있는 전략 컨텍스트"""

class KNNImputerStrategy(ImputationStrategy):
    """K-근접 이웃 방식을 사용하는 데이터 대치"""